In [1]:
import numpy as np
import pandas as pd
#import re
#import os
#import operator
#import math
import pickle
import xlrd

In [2]:
# to use functions in another notebook
#import ipynb
#import matplotlib.pylab as plt

In [3]:
import pulp as pulp
from pulp import *

In [4]:
#%run Build_Optimization_Database.ipynb
#df = calc_impacts('CH','aug')

In [58]:
dftest=pd.read_pickle('CH_aug.p')

## Optimization

In [6]:
impactperpill = 0.31 # kgCO2eq per pill

In [7]:
def editdf(df,impact,preference,supplement):
    
    df.at[(df['Food Name'].str.contains('pumpkin'))&(df['Food Name'].str.contains('boiled')),'Food Name'] = 'squash, pumpkin, boiled'
    df.at[(df['Food Name'].str.contains('pumpkin'))&(df['Food Name'].str.contains('raw')),'Food Name'] = 'squash, pumpkin, raw'
    remove = ['AN','OE','AM','BP','AO','SEC','DAP','SN','WCGY','WCG','WCN','Q','SEA','SEC',\
              'SN','WCG','WCN','AA','AA_wg','PAC','SC' ]
    df = df[~df['Group'].isin(remove)]
    df = df[~df['Food Name'].str.contains('ice cream')]
    df = df[~df['Food Name'].str.contains('cocoa')]
    df = df[~df['Food Name'].str.contains('raisins')]
    # remove lemons limes
    sub = df[((df['Food Name'].str.contains('lemon|lime'))&(df['Group']=='FAT'))].index.tolist()
    df.drop(sub,axis=0,inplace=True)
    
    sub1 = df.loc[(df['Food Name'].str.contains('peas|beans|lentils'))&(df['Food Name'].str.contains('raw')\
        &(df['Food Name'].str.contains('dried'))&(~df['Food Name'].str.contains(\
                                'frozen|sugar|runner|cluster|sprouts|canned')))].index.tolist()
    df.drop(sub1,axis=0,inplace=True)
    sub2 = df.loc[(df['Food Name'].str.contains('tomato puree'))].index.tolist()
    df.drop(sub2,axis=0,inplace=True)
    
    df = df.fillna(0)
    df['Food Name']= df[['Food Name','Group']].apply(lambda x: '_'.join(x),axis=1)
    df = df[~((df['Group'].str.startswith('M'))&((df['Group'].str.endswith('R'))))]    
    
    # remove raw meat () and raw fish
    # remove cooked vegetables (DGC)
    #df = df[~(df['Group']=='DGC')]

    df = df[~((df['Group'].str.startswith('J'))&(df['Group'].str.endswith('R')))]
    
    if preference == 'vegan':
        #fish,meat,dairy
        df = df[~(df['Group'].str.startswith('J')|df['Group'].str.startswith('M'))]
        df = df[~(df['Group'].isin(['BLS','BLH','BLF','BN','BAH','BAK','BAE']))]
        df = df[~(df['Group']=='CA')]#butter
        df = df[~(df['Group']=='OA')]#eggs
        df = df[~df['Food Name'].str.contains('egg,')]
        df = df[~df['Food Code'].isin(['11-795','11-793','11-719'])] # get rid of oats with milk and egg noodles
    elif preference == 'vegetarian':
        df = df[~(df['Group'].str.startswith('J')|df['Group'].str.startswith('M'))]
    elif preference == 'pescatarian':
        df = df[~df['Group'].str.startswith('M')]
    else:pass
    
    if supplement == 'yes_sup':
        sup_dict = {'Calcium (mg)':1200,'Vitamin B12 (µg)':3,'Iron (mg)':18,\
        'Zinc (mg)':10,'Selenium (µg)':55,'Retinol Equivalent (µg)':900,'Vitamin D (µg)':20,\
        'Iodine (µg)':150, 'Vitamin K1 (µg)':120,'Food Name':'supplement','Group':'SUP',\
        'optimization_value_GHG_1_trade':impactperpill,'GHG_bioopt_value1':impactperpill,
         'optimization_value_BIO_1_trade':5.5e-16,'bio_GHGopt_value1':5.5e-16,
            'conversion factor':1,'Edible proportion':1,\
                    }     
        df = df.append(sup_dict,ignore_index=True)
        df = df.fillna(0)
        #np.mean(df1['global_landbio_gram'])*10 for value for biodiversity loss from a supplement
    
    else:pass
    
    
    # these need to be updated in next version to account for no trade and no home production
    df = df[(df['trade_impacts_GHG'].str.len()!=0)|(df['trade_impacts_BIO'].str.len()!=0)]
    df = df[df['optimization_value_GHG_1_trade']!=0]
    df = df[df['optimization_value_BIO_1_trade']!=0]
    df = df[(df['bio_GHGopt_value1']!=0)|(df['GHG_bioopt_value1']!=0)]

    # additional impact due to not all parts being consumed:
    # take non-edible proportion, multiply by impact per gram of total product, add this to edible proportion impact
    # because even though this isn't consumed as part of the grams recommended, it is still produced. Need to 
    # say non-edible proportion is 1 (not 0) if whole thing is consumed to avoid getting zero impact
    m1 = (df['Edible proportion']!=1)
    df_valid = df[m1]
    df.loc[m1,'Non_edible'] = 1 - df_valid['Edible proportion']
    
    for i in ['optimization_value_GHG_1_trade','bio_GHGopt_value1','optimization_value_BIO_1_trade',\
              'GHG_bioopt_value1']:
        df[i] = df[i]*df['conversion factor']
        df.loc[m1,i] = df.loc[m1,i]+(df.loc[m1,i]*df.loc[m1,'Non_edible'])
    df.reset_index(inplace=True,drop=True)   
    return df

In [8]:
def filter(foodgroup,variable):
    filtered = {k:v for (k,v) in variable.items() if any('_'+q in k for q in foodgroup)}
    return filtered

In [9]:
def fruitlimits(df,numdays):
    fruit_total = 250*numdays # grams per day
    fruit_groups =('FAT','FAT')
    fruit_serv_min = 75
    return fruit_total,fruit_groups,fruit_serv_min

In [10]:
def veglimits(df,numdays):
    vegetable_total = 360*numdays # grams per day
    vegetable_groups = ('DGR','DGC')
    vegetable_serv_min = 50 ## largest serving size of vegetables
    return vegetable_total,vegetable_groups,vegetable_serv_min

In [11]:
def legumelimits(df,numdays):
    legume_total = 60*numdays # grams per day
    legume_serv_min = 50 ## largest serving size of legumes
    legume_groups = ('DBR','DBC','DFR','DFC')
    return legume_total,legume_groups,legume_serv_min

In [12]:
def grainlimits(df,numdays):
    grains_total = 125*numdays # grams per day
    grains_groups = ('AA_wg','AC_wg','AD_wg','AI_wg','AF_wg')
    grains_num = 3
    pasta_rice_pot_serve_min = 50
    pasta_rice_group = ['AC','AC_wg','DAR','DAC']
    cereal_grain_bread_serve_min = 30
    cereal_grain_bread_group = ['AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg']
    return (grains_total,grains_groups,pasta_rice_pot_serve_min,pasta_rice_group,cereal_grain_bread_serve_min,
        cereal_grain_bread_group)

In [13]:
def nutslimits(df,numdays):
    nuts_total = 21*numdays # grams per day
    nuts_serv_min = 10 ## largest serving size of nuts
    nuts_groups = ('GAT','GAT')  
    return nuts_total,nuts_groups,nuts_serv_min

In [14]:
def milklimits(df,numdays):
    milk_total = 435*numdays # grams per day
    milk_serv_min = 50
    milk_groups = ('BAE','BAH','BAK')
    return milk_total,milk_groups,milk_serv_min
    

In [15]:
def cheeselimits(df,numdays):
    soft_cheese_serve_min = 50
    soft_group =['BLS']
    med_cheese_serve_min = 40
    med_group =['BLM']
    hard_cheese_serve_min = 20
    hard_group =['BLH']
    fresh_cheese_serve_min = 50
    fresh_group =['BLF']
    yogurt_serve_min = 50
    yogurt_group =['BN']
    return (soft_cheese_serve_min,soft_group,med_cheese_serve_min,med_group,hard_cheese_serve_min,hard_group,
       fresh_cheese_serve_min,fresh_group, yogurt_serve_min,yogurt_group)

In [16]:
def milksublimits(df,numdays):
    milksub_serve_min = 50
    milksub_group =['BAV']
    yogsub_serve_min = 50
    yogsub_group = ['BNV']
    return milksub_serve_min,milksub_group,yogsub_serve_min,yogsub_group

In [17]:
def veganlimits(df,numdays):
    vegan_serve_min = 100
    vegan_group =['VEG']

    return vegan_serve_min,vegan_group

In [18]:
def meatlimits(df,numdays):
    fish_serve_min = 50
    fish_groups = df[df['Group'].str.startswith('J')]['Group'].tolist()
    
    red_meat_max = 22.5*numdays
    red_meat_groups = ['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
                       'MAIR','MAIC']
    red_serve_min = 22.5

    whitemeat_serve_min = 50
    white_meat_groups = ['MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC']
    
    processed_min = 2
    processed_total = 2*numdays # grams per day
    processed_groups = ['MI','MAAC','MAAR']

    oil_serve_min = 14
    oil_num = 2
    oil_min = oil_serve_min-oil_serve_min*0.1
    oil_max = oil_serve_min+oil_serve_min*0.1
    oil_groups = df[(df['Group'].str.startswith('O'))|(df['Group']=='BTM')|(df['Group']=='BJC')]['Group'].tolist()

    egg_serve_min = 60 # this assumes one large egg at 60 grams each 
    egg_groups = df[df['Group']=='CA']['Group'].tolist()
    return (fish_serve_min,fish_groups,red_meat_max,red_meat_groups,red_serve_min,whitemeat_serve_min,
        white_meat_groups,processed_total,processed_min,processed_groups,oil_serve_min,oil_min,oil_max,oil_groups,
            egg_serve_min,egg_groups)

In [19]:
def nutrient(gender,age,numdays):
    df_nut = pd.read_excel('7f_to7_nutrients.xlsx')
    names = df_nut.columns.tolist()[2:]
    mass = [i.split(')',1)[0].split('(')[1] for i in df_nut.loc[0].tolist()[2:]]
    ## calculate the b inequality values for optimization based on user input   
    if (gender==0 and age<=30):
        nut_limits = [i for i in df_nut.loc[10]][2:]
    elif (gender==0 and age>30 and age<51):   
        nut_limits = [i for i in df_nut.loc[11]][2:]
    elif (gender==0 and age>51 and age<70):   
        nut_limits = [i for i in df_nut.loc[12]][2:]
    elif (gender==0 and age>70):   
        nut_limits = [i for i in df_nut.loc[13]][2:]
    elif (gender==1 and age<=30):
        nut_limits = [i for i in df_nut.loc[17]][2:]
    elif (gender==1 and age>30 and age<51):   
        nut_limits = [i for i in df_nut.loc[18]][2:]
    elif (gender==1 and age>51 and age<70):   
        nut_limits = [i for i in df_nut.loc[19]][2:]
    elif (gender==1 and age>70): 
        nut_limits = [i for i in df_nut.loc[20]][2:]
    else: ## default woman between 31 and 50
        nut_limits = [i for i in df_nut.loc[18]][2:]
    
    return [(j,round(i*numdays,2),m) for i,j,m in zip(nut_limits,names,mass)]

In [20]:
def personal(gender,weight,height,age,actlevel,numdays):
    age = age
    weight = weight ## in kg
    height = height ## in cm   
    
    activity = PAL(actlevel)
   
    if gender == 0:
        kcal = (10*weight + 6.25*height - 5*age - 161)*activity*numdays
    else:
        kcal = (10*weight + 6.25*height - 5*age + 5)*activity*numdays
    
    nut_limits = nutrient(gender,age,numdays)
    
    return kcal,nut_limits
    

In [21]:
# The PAL of a subject can be classified in three categories as defined by the last FAO/WHO/UNU 
# expert consultation on human energy requirements (2004). The physical activity for sedentary and 
# light activity lifestyles ranges between 1.40 and 1.69, for moderately active or active lifestyles 
# between 1.70 and 1.99, and for vigorously active lifestyles between 2.00 and 2.40.
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3636460/

In [22]:
def PAL(lifestyle):
    if lifestyle == 'sedentary':
        activity = 1.4
    elif lifestyle == 'moderate':
        activity = 1.7
    elif lifestyle == 'vigorous':
        activity = 2.0
    else: ## default
        activity = 1.4
    return activity

In [23]:
df=dftest
impact = 'GHG'
gender = 0
weight = 60
height = 165
age = 40
actlevel ='moderate'
location ='CH'
month = 'aug'
numdays = 7
preference = 'none'
relax =2
supplement ='yes_sup'

In [24]:
df = editdf(df,impact,preference,supplement) # get the right dataframe and edit it for optimization

In [25]:
kcal,nut_limits = personal(gender,weight,height,age,actlevel,numdays) # get the right kcal and nutrients

In [26]:
fruit_total, fruit_groups, fruit_serv_min = fruitlimits(df,numdays)
vegetable_total,vegetable_groups,vegetable_serv_min = veglimits(df,numdays)
legume_total,legume_groups,legume_serv_min = legumelimits(df,numdays)
grains_total,grains_groups,pasta_rice_pot_serve_min,pasta_rice_group,cereal_grain_bread_serve_min,\
        cereal_grain_bread_group = grainlimits(df,numdays)
nuts_total,nuts_groups,nuts_serv_min = nutslimits(df,numdays)
milk_total,milk_groups,milk_serv_min = milklimits(df,numdays)
soft_cheese_serve_min,soft_group,med_cheese_serve_min,med_group,hard_cheese_serve_min,hard_group,\
       fresh_cheese_serve_min,fresh_group, yogurt_serve_min,yogurt_group = cheeselimits(df,numdays)
fish_serve_min,fish_groups,red_meat_max,red_meat_groups,red_serve_min,whitemeat_serve_min,\
        white_meat_groups,processed_total,processed_min,processed_groups,oil_serve_min,oil_min,oil_max,oil_groups,\
            egg_serve_min,egg_groups = meatlimits(df,numdays)  
vegan_serve_min,vegan_group = veganlimits(df,numdays)
milksub_serve_min,milksub_group,yogsub_serve_min,yogsub_group = milksublimits(df,numdays)

In [27]:
PUFA = 0.11 # more than 11% of energy should come from polyunsaturated fatty acids
transFA = 0.005 #less than 0.5% of kcal per day should come from trans fats
omega3 = 250*numdays # mg per day 

In [28]:
nutrient_source = ['Calcium (mg)','Copper (mg)', 'Iodine (µg)', 'Iron (mg)', 'Magnesium (mg)','Manganese (mg)',
        'Phosphorus (mg)', 'Selenium (µg)','Zinc (mg)','Potassium (mg)', 'Sodium (mg)','Chloride (mg)', 
        'Retinol Equivalent (µg)','Vitamin C (mg)', 'Vitamin D (µg)', 'Vitamin E (mg)', 'Vitamin K1 (µg)', 'Thiamin (mg)',
        'Riboflavin (mg)', 'Niacin equivalent (mg)', 'Vitamin B6 (mg)', 'Folate (µg)', 'Vitamin B12 (µg)',
        'Pantothenate (mg)','Biotin (µg)', 'Carbohydrate (g)','AOAC fibre (g)', 'Fat (g)','Protein (g)']

#nutrient_lim = ['Calcium', 'Copper', 'Iodine', 'Iron', 'Magnesium', 'Manganese', 'Phosphorus',\
#    'Selenium', 'Zinc','Potassium', 'Sodium', 'Chloride', 'Vitamin A', 'Vitamin C', 'Vitamin D',\
#    'Vitamin E', 'Vitamin K', 'Thiamin','Riboflavin', 'Niacin', 'Vitamin B6', 'Folate', 'Vitamin B12',\
#    'Pantothenic Acid', 'Biotin', 'Carbohydrate', 'Total Fiber', 'Fat', 'Proteinb'] 

In [29]:
prob = LpProblem('Optimization',LpMinimize)
    # Create a list of the foods 
foodgroups=list(df['Food Name'])
food_status = LpVariable.dicts("food_status",(foodgroup for foodgroup in foodgroups),cat='Binary')        
x = {foodamount:LpVariable(foodamount, lowBound=0) for foodamount in foodgroups}  

In [30]:
nut_lim_noVD =[i[1] for i in nut_limits if 'Vitamin D' not in i[0]]
nut_source_noVD = [i for i in nutrient_source if 'Vitamin D' not in i]
for nutlimit,nutrient in zip(nut_lim_noVD,nut_source_noVD):
    prob += (lpSum([df[df['Food Name']==i][nutrient]*x[i]for i in foodgroups])>=nutlimit,
             'Nutrient Constraint: %s%s'%(nutlimit,nutrient))

In [31]:
# include Vit D
prob += (lpSum([df[df['Food Name']==i]['Vitamin D (µg)']*x[i]for i in foodgroups])>=[i[1] for i in nut_limits if i[0]=='Vitamin D'][0],
                       'Nutrient Constraint: %s%s'%(nutlimit,'Vitamin D (µg)'))

In [32]:
if impact == 'GHG':
    impactsnew = dict(zip(foodgroups,df['optimization_value_GHG_1_trade'])) # objective 1
    impactlim = dict(zip(foodgroups,df['bio_GHGopt_value1']))
    prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total Impacts per person"
elif impact == 'BIO':
    impactsnew = dict(zip(foodgroups,df['optimization_value_BIO_1_trade'])) # objective 2
    impactlim = dict(zip(foodgroups,df['GHG_bioopt_value1']))
    prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total Impacts per person"
else:   # this needs to be changed         
    impactsnew = dict(zip(foodgroups,df['optimization_value_GHG_1_trade'])) # objective 1
    impactlim = dict(zip(foodgroups,df['bio_GHGopt_value1']))
    prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total Impacts per person"

In [33]:
        # kcal
prob += (lpSum([df[df['Food Name']==i]['Energy (kcal) (kcal)']*x[i]for i in foodgroups])>=kcal,'kcal')
        
        #sodium
prob += (lpSum([df[df['Food Name']==i]['Sodium (mg)']*x[i]for i in foodgroups])<=2300*numdays,'sodium')

In [34]:
for i,j in zip(foodgroups,food_status):
    prob += food_status[j]*1<=x[i],'Binary Constraint_1: %s'%i # if xi == 0, food status i is allowed
            # to be 0. when xi > 0, food_status i is forced to flip to 1 in order to raise the upper bound to M.
    prob += x[i]<=food_status[j]*100000, 'Binary Constraint_2: %s'%i

In [35]:
for limit,group in zip([fruit_total,vegetable_total,legume_total,grains_total,nuts_total],
               [fruit_groups,vegetable_groups,legume_groups,grains_groups,nuts_groups]):
    prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
                     'GBD Constraint: %s%s'%(limit,group))
    
prob += (lpSum([filter(processed_groups,x)[i] for i in filter(processed_groups,x)])<=processed_total,\
                     'GBD Constraint: %s%s'%(processed_total,processed_groups))
     
        
if preference == 'none': 
    for limit,group in zip([milk_total],[milk_groups]):
        prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
                         'GBD Constraint: %s%s'%(limit,group))
            
elif preference == 'vegetarian':
    for limit,group in zip([milk_total],[milk_groups]):
        prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
                         'GBD Constraint: %s%s'%(limit,group))

elif preference == 'vegan':
    pass      
elif preference == 'nomilk':
    pass
else:
    pass

In [36]:
        # nutrient constraints from GBD
prob += (lpSum([df[df['Food Name']==i]['Omega3_all']*x[i] for i in foodgroups])>= omega3,\
                 'Omega3 constraints')
prob += (lpSum([df[df['Food Name']==i]['Trans FA_kcal']*x[i] for i in foodgroups])<= transFA*kcal,\
                 'TFA constraints')
prob += (lpSum([df[df['Food Name']==i]['PUFA_kcal_POLY_DB']*x[i] for i in foodgroups])>= PUFA*kcal,\
                 'PUFA constraints') 

In [37]:
listofcheckboxes = {'fruits':['FAT'],'vegetables':['DGR','DGC'],
     'legumes':['DBR','DBC','DFR','DFC'],'grains':['AC','AC_wg','DAR','DAC',
     'AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg'],'nuts_seeds':
     ['GAT'],'milk':['BAE','BAH','BAK','BLS','BLM','BLH','BLF','BN'],
     'vegetarianproducts':['BAV','BNV','VEG'],
     'fish':['JAC', 'JAR', 'JCC', 'JCR', 'JKC', 'JKR', 'JMC', 'JMR', 'JRC'],
    'meat':['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
    'MAIR','MAIC','MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC',
    'MI','MAAC','MAAR'],'eggs_oil':['CA','OA', 'OC', 'OE','BTM','BJC']}

In [38]:
# keys are user inputs from Userinput page
mustincludedict = \
{'beans': {'mass': 100, 'names': 'beans, blackeye, whole, dried, raw, canned_DBR'},
 'blueberries': {'mass': 400, 'names': 'blueberries, frozen_FAT'},
 'broccoli': {'mass': 1100,
  'names': 'broccoli, green, boiled in salted water_DGC'},
 'butter': {'mass': 50, 'names': 'butter, salted_OA'}}
# 'cheese': {'mass': 400, 'names': 'cheese, caerphilly_BLH'},
# 'oil': {'mass': 50, 'names': 'oil, coconut_OC'},
# 'rabbit': {'mass': 650, 'names': 'rabbit, raw, meat only'},
# 'tortilla': {'mass': 550, 'names': 'tortilla, wheat, soft_AF'}}
# 'venison': {'mass': 550, 'names': 'venison, roast'}}

In [39]:
for key,value in mustincludedict.items():
    prob += (lpSum({k:v for (k,v) in x.items() if value['names'] in k})>=value['mass'])#,'personal preferences_%s%s'%(key,value['names']) 

In [40]:
for serve,i in zip([fruit_serv_min,vegetable_serv_min,legume_serv_min,nuts_serv_min,milk_serv_min],
                [fruit_groups,vegetable_groups,legume_groups,nuts_groups,milk_groups]):
    for k,l in zip(filter(i,x),filter(i,food_status)):
        prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                    'Min Serving Size Limits: %s%s%s'%(serve,i,k)
            # dairy product min limits
for serve,i in zip([soft_cheese_serve_min,med_cheese_serve_min,hard_cheese_serve_min,
                                fresh_cheese_serve_min,yogurt_serve_min],
                               [soft_group,med_group,hard_group,fresh_group,yogurt_group]):
    for k,l in zip(filter(i,x),filter(i,food_status)):
        prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                    'Min Serving Size Limits: %s%s%s'%(serve,i,k)

for serve,i in zip([milksub_serve_min,yogsub_serve_min,vegan_serve_min],
                               [milksub_group,yogsub_group,vegan_group]):
    for k,l in zip(filter(i,x),filter(i,food_status)):
        prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                    'Min Serving Size Limits: %s%s%s'%(serve,i,k)        
           # grain product min limits
for serve,i in zip([pasta_rice_pot_serve_min,cereal_grain_bread_serve_min],
                               [pasta_rice_group,cereal_grain_bread_group]):
    for k,l in zip(filter(i,x),filter(i,food_status)):
        prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                    'Min Serving Size Limits: %s%s%s'%(serve,i,k) 
# meat,egg, oil, fish product min limits      
for serve,i in zip([fish_serve_min,red_serve_min,whitemeat_serve_min,oil_serve_min,egg_serve_min],
                               [fish_groups,red_meat_groups,white_meat_groups,oil_groups,egg_groups]):
    for k,l in zip(filter(i,x),filter(i,food_status)):
        prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                    'Min Serving Size Limits: %s%s%s'%(serve,i,k)   

In [41]:
#flat_list = [item for sublist in listofnamestoinclude for item in sublist]

In [43]:
def maxfoodlimits(foodgroup,minsize):
    global prob
    for k,l in zip(filter(foodgroup,x),filter(foodgroup,food_status)):
        if k not in [v['names'] for k,v in mustincludedict.items()]:
            prob += filter(foodgroup,x)[k] <= relax*minsize*numdays*\
                      filter(foodgroup,food_status)[l]
        else:
            #print (k)
            prob += filter(foodgroup,x)[k] <= [v1['mass']*1.1 for k1,v1 in mustincludedict.items() if k in v1['names']][0]*\
                       filter(foodgroup,food_status)[l]      
    #print (prob)
    return prob

In [44]:
maxfoodlimits(fruit_groups,fruit_serv_min)
maxfoodlimits(vegetable_groups,vegetable_serv_min)
maxfoodlimits(pasta_rice_group,pasta_rice_pot_serve_min)
maxfoodlimits(cereal_grain_bread_group,cereal_grain_bread_serve_min)
maxfoodlimits(legume_groups,legume_serv_min)
maxfoodlimits(nuts_groups,nuts_serv_min)
maxfoodlimits(fish_groups,fish_serve_min)
maxfoodlimits(white_meat_groups,whitemeat_serve_min)
maxfoodlimits(oil_groups,oil_serve_min)
maxfoodlimits(egg_groups,egg_serve_min)
maxfoodlimits(red_meat_groups,red_serve_min)
maxfoodlimits(processed_groups,processed_min)
maxfoodlimits(vegan_group,vegan_serve_min)
maxfoodlimits(milksub_group,milksub_serve_min)
maxfoodlimits(yogsub_group,yogsub_serve_min)
maxfoodlimits(yogurt_group,yogurt_serve_min)
maxfoodlimits(fresh_group,fresh_cheese_serve_min)
maxfoodlimits(soft_group,soft_cheese_serve_min)
maxfoodlimits(med_group,med_cheese_serve_min)
maxfoodlimits(hard_group,hard_cheese_serve_min)
print()

In [45]:
for k,l in zip(filter(milk_groups,x),filter(milk_groups,food_status)):
    prob += filter(milk_groups,x)[k] <= 1000*numdays*\
                        filter(milk_groups,food_status)[l],('Max Limits_BAX: %s%s'%(k,l))

In [46]:
for cheese_type,min_serve in zip([fresh_group,soft_group,med_group,hard_group,yogurt_group],\
                        [fresh_cheese_serve_min,soft_cheese_serve_min,med_cheese_serve_min,\
                         hard_cheese_serve_min,yogurt_serve_min]):      
    for k,l in zip(filter(cheese_type,x),filter(cheese_type,food_status)):
        prob += filter(cheese_type,x)[k] <= relax*min_serve*numdays*\
                        filter(cheese_type,food_status)[l],('Max Limits_BXX: %s%s%s%s'%(k,l,cheese_type,min_serve))

In [47]:
#(4 grams per clove, 5 cloves per day)  
prob += (lpSum({k:v for (k,v) in x.items() if 'garlic' in k})<=4*5*numdays),('Max Limits garlic')

In [48]:

## can't choose same food item more than 1 time (i.e. fresh and canned peaches)    
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if 'FAT' in k})):
    subset0 = {k:v for (k,v) in food_status.items() if k.startswith(i) and 'FAT' in k}
    prob += lpSum(subset0)<=1,'Food Overlap Constraint_FAT%s'%i 
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if 'GAT' in k})):
    subset5 = {k:v for (k,v) in food_status.items() if k.startswith(i) and 'GAT' in k}
    prob += lpSum(subset5)<=1,'Food Overlap Constraint_GAT%s'%i
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_AI' in k})):
    subset6 = {k:v for (k,v) in food_status.items() if i in k}
    prob += lpSum(subset6)<=1,'Food Overlap Constraint_AI%s'%i 
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DF' in k})):
    subset7 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DF',i])}
    prob += lpSum(subset7)<=1,'Food Overlap Constraint_DF%s'%i         
for i in list(set({k.split(',')[1] for (k,v) in food_status.items() if '_DB' in k})):
    subset8 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DB',i])}
    prob += lpSum(subset8)<=1,'Food Overlap Constraint_DB%s'%i       
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DG' in k})):
    subset9 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DG',i])}
    prob += lpSum(subset9)<=1,'Food Overlap Constraint_DG%s'%i
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DA' in k})):
    subset15 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DA',i])}
    prob += lpSum(subset15)<=1,'Food Overlap Constraint_DA%s'%i        
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_J' in k})):
    subset10 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_J',i])}
    prob += lpSum(subset10)<=1,'Food Overlap Constraint_J%s'%i 
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_BA' in k})):
    subset11 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_BA',i])}
    prob += lpSum(subset11)<=1,'Food Overlap Constraint_BA%s'%i          

for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_CA' in k})):
    subset12 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_CA',i])}
    prob += lpSum(subset12)<=1,'Food Overlap Constraint_CA%s%s'%(i,k)                
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_BAV' in k})):
    subset13 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_BAV',i])}
    prob += lpSum(subset13)<=1,'Food Overlap Constraint_BAV%s%s'%(i,k)                             
for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_AF' in k})):
    subset14 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_AF',i])}
    prob += lpSum(subset14)<=1,'Food Overlap Constraint_AF%s%s'%(i,k)            
           
subset1 = {k:v for (k,v) in food_status.items() if '_BTM' in k}
prob += lpSum(subset1)<=1,'Food Overlap Constraint%s'%'BTM'
subset2 = {k:v for (k,v) in food_status.items() if '_OA' in k}
prob += lpSum(subset2)<=1,'Food Overlap Constraint%s'%'OA'
subset3 = {k:v for (k,v) in food_status.items() if '_OC' in k}
prob += lpSum(subset3)<=2,'Food Overlap Constraint%s'%'OC'

In [49]:
prob.writeLP('Optimization.lp')
prob.solve()

1

In [50]:
            
#answer = value(prob.objective)
#print("Total Impacts per person = ", value(prob.objective))
print("Status:", LpStatus[prob.status])
results = []
            # Each of the variables is printed with its resolved optimum value
for v in prob.variables():
    re=[v.name,v.varValue]
    results.append(re)

Status: Optimal


In [51]:
results_food = [i for i in results if 'food_status' not in i[0]]

In [52]:
results_food = [i for i in results_food if 'dummy' not in i[0]]

In [53]:
results_food_dict = {item[0]:item[1] for item in results_food}

In [54]:
df['Food Name'] = [i.replace(' ','_') for i in df['Food Name']]

In [55]:
df['Grams'] = df['Food Name'].map(results_food_dict)

In [56]:
df_final = df[['Food Name','Grams','optimization_country_GHG_1_trade','optimization_value_GHG_1_trade',
    'bio_GHGopt_value1', 'optimization_value_BIO_1_trade','optimization_country_BIO_1_trade',
               'GHG_bioopt_value1','bio_GHGopt_value1']]
df_final[df_final['Grams']!=0]

,Food Name,Grams,optimization_country_GHG_1_trade,optimization_value_GHG_1_trade,bio_GHGopt_value1,optimization_value_BIO_1_trade,optimization_country_BIO_1_trade,GHG_bioopt_value1,bio_GHGopt_value1
16,"okra,_stir-fried_in_corn_oil_DGC",NaN,CH,0.004979,8.223796e-18,1.455507e-18,IT,0.005068,8.223796e-18
18,"baking_fat_and_margarine_(75-90%_fat),_hard_bl...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
19,"fat_spread,_low_fat_(26-39%),_not_polyunsatura...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
20,"fat_spread,_low_fat_(26-39%),_not_polyunsatura...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
21,"fat_spread,_low_fat_(26-39%),_polyunsaturated_BTM",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
22,"fat_spread,_reduced_fat_(41-62%),_not_polyunsa...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
23,"fat_spread,_reduced_fat_(41-62%),_not_polyunsa...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
24,"fat_spread,_reduced_fat_(41-62%),_polyunsatura...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
25,"fat_spread,_reduced_fat_(62-75%),_not_polyunsa...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18
26,"fat_spread,_reduced_fat_(62-75%),_polyunsatura...",NaN,CH,0.006899,4.364487e-18,2.939663e-18,BE,0.007008,4.364487e-18


In [57]:
print (df_final.head(2))

                        Food Name  Grams optimization_country_GHG_1_trade  \
0  almonds,_flaked_and_ground_GAT    0.0                               US   
1            almonds,_toasted_GAT    0.0                               US   

   optimization_value_GHG_1_trade  bio_GHGopt_value1  \
0                        0.003466       2.132229e-17   
1                        0.003466       2.132229e-17   

   optimization_value_BIO_1_trade optimization_country_BIO_1_trade  \
0                    1.124133e-17                               AU   
1                    1.124133e-17                               AU   

   GHG_bioopt_value1  bio_GHGopt_value1  
0           0.006993       2.132229e-17  
1           0.006993       2.132229e-17  
